In [ ]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from sklearn.model_selection import train_test_split
from torch import nn, optim
import matplotlib.pyplot as plt

import copy

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def split_array(arr, n):
    avg = len(arr) // n
    remainder = len(arr) % n
    result = []
    start = 0

    for i in range(n):
        if i < remainder:
            end = start + avg + 1
        else:
            end = start + avg
        result.append(arr[start:end])
        start = end

    return result

In [ ]:
def creat_data(n):
    data= pd.read_csv("mail.csv")
    
    
    

    trainloaders=[]
    testloaders = []


        


    
    
    
    X = data['Message']
    y = data['real']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)
    X_train = feature_extraction.fit_transform(X_train)

    X_test = feature_extraction.transform(X_test)

    X_train=X_train.toarray()
    X_test=X_test.toarray()



    X_train=split_array(X_train, n)
 
    y_train=split_array(y_train, n)


    X_test = torch.Tensor(X_test)
    y_test= torch.Tensor(y_test.values)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    X_test=X_test.to(device)
    y_test=y_test.to(device)
    X_test_hold=X_test
    y_test_hold=y_test
    testloaders=[]
    testloaders.append((X_test,y_test))
    for i in range(n):
        X_train[i] = torch.Tensor(X_train[i])
        y_train[i] = torch.Tensor(y_train[i].values)
        X_train[i] = torch.tensor(X_train[i], dtype=torch.float32)
        y_train[i] = torch.tensor(y_train[i], dtype=torch.float32).reshape(-1, 1)

        X_train[i]=X_train[i].to(device)
        y_train[i]=y_train[i].to(device)



        
        hold1=[]
        hold2=[]
        hold1.append((X_train[i],y_train[i]))
        trainloaders.append(hold1)
    
    
    
    combined_testloader = [item for sublist in testloaders for item in sublist]

    return trainloaders,combined_testloader,X_test_hold,y_test_hold

In [ ]:
# model definition
class ModelA(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden2 = nn.Linear(100, 200)
        self.act2 = nn.Sigmoid()
        self.hidden3 = nn.Linear(200, 50)
        self.output= nn.Linear(50, 1)

        self.act_output = nn.Sigmoid()
        


 
    def forward(self, x):
        #x = self.act2(self.input_layer(x))
        x = self.act2(self.hidden2(x))
        x = self.act2(self.hidden3(x))
        #x = self.act_output(self.output(x))

        return x








In [ ]:
class ModelB(nn.Module):
    def __init__(self, model):
        super(ModelB, self).__init__()

        self.input_layer = nn.Linear(1894, 100)
        self.act1 = nn.Sigmoid() 
        self.model = model# Insert Model A in between input and output
        
        self.output= nn.Linear(50, 1)

        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.input_layer(x))
        x = self.model(x)  # Pass through Model A
        x = self.act_output(self.output(x))
        return x
    
    def give(self):
        return self.model

In [ ]:
def train(model,n_epochs,batch_size,traindata,lr):
    X_train=traindata[0][0]
    model=model.to(device)
    y_train=traindata[0][1]
    loss_func = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    for epoch in range(n_epochs):
            for i in range(0, len(X_train), batch_size):
                Xbatch = X_train[i:i+batch_size]
                y_pred = model(Xbatch)
                ybatch = y_train[i:i+batch_size]
                loss = loss_func(y_pred, ybatch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                

            if epoch % 500 == 0 & epoch != 0:
                with torch.no_grad():
                    y_pred = model(X_train)

                accuracy = (y_pred.round() == y_train).float().mean()
                with torch.no_grad():
                    y_pred = model(X_train)



                print(f"Accuracy {accuracy}")
                print(f'Finished epoch {epoch}, latest loss {loss}')


def test(model, testloader):
    """Evaluate the network on the entire test set."""
    model=model.to(device)
    X_test=testloader[0][0]
    
    y_test=testloader[0][1]
    correct, total, loss = 0, 0, 0.0
    loss_func = nn.MSELoss()
    with torch.no_grad():
        y_pred = model(X_test)
    accuracy = (y_pred.round() == y_test).float().mean()

    loss = loss_func(y_pred, y_test)

    return loss.tolist(), accuracy.tolist()
    


def test_whole_data(model, x,y):
    """Evaluate the network on the entire test set."""
    model=model.to(device)
    X_test=x
    y_test=y
    correct, total, loss = 0, 0, 0.0
    loss_func = nn.MSELoss()
    with torch.no_grad():
        y_pred = model(X_test)
    accuracy = (y_pred.round() == y_test).float().mean()

    loss = loss_func(y_pred, y_test)

    return loss.tolist(), accuracy.tolist()

In [ ]:
def num_model(n):
    model_list=[]
    for i in range(n):
        model1=ModelA()


        model2=ModelB(model1)
     
        model_list.append(model2)
    return model_list


In [ ]:
def train_global_standard(n,k,n_epochs,batch_size):

    trainloaders,testloaders,x_test,y_test=creat_data(n)
    

    mod_list=num_model(n)
    

    max_value=0

    acc=[]
    epo=[]
    curr=0
    for i in range(k):

        for z in range(n):
            train(mod_list[z],n_epochs,batch_size,trainloaders[z],lr=0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])  
        if avg_value > max_value:
             max_value=avg_value


        parameter_lists=[]
        for model in mod_list:
            hold=model.give()
            parameter_lists.append(hold.parameters())





        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        

        glob_hold = ModelA() 

        
        updated_params = []

        
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

       
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)

        for model_instance in mod_list:
                model_instance.model = glob_hold


    
    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with 15 Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()
    return max_value





In [ ]:
train_global_standard(15,500,5,32)
    

In [ ]:
def train_global_imp(n,k,n_epochs,batch_size):

    trainloaders,testloaders,x_test,y_test=creat_data(n)
    


    
    

    mod_list=num_model(n)
    for i in range(n):
        train(mod_list[i],1000,batch_size,trainloaders[i],lr=0.001)
    
        

    
    

    



    acc=[]
    epo=[]
    curr=0
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,batch_size,trainloaders[i],lr=0.002)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])    


        parameter_lists=[]
        k=0
        for model in mod_list:
            if res_acc[k]>0:
                print("fedder")
                hold=model.give()
                parameter_lists.append(hold.parameters())
            k=k+1





        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        

        glob_hold = ModelA() 

        
        updated_params = []

        
        for glob_param, avg_param in zip(glob_hold.parameters(), avg_params):
            updated_params.append(avg_param)

       
        for glob_param, updated_param in zip(glob_hold.parameters(), updated_params):
            glob_param.data.copy_(updated_param)

        # Freeze all layers in glob_hold
        if curr % 32 == 0:
            print("fixer")
            
            for param in glob_hold.parameters():
                param.requires_grad = False
        

            for model_instance in mod_list:
                model_instance.model = glob_hold
                

      
            k=0
            for i in range(n):
                if res_acc[k]<avg_value:
                    print("fed_fix")
                    train(mod_list[i],n_epochs+50,batch_size,trainloaders[i],lr=0.0035)
                k=k+1

            for model_instance in mod_list:
                    for param in model_instance.parameters():
                        param.requires_grad = True
        else:
            for model_instance in mod_list:
                model_instance.model = glob_hold




        

         
            







        



    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with 15 Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()






In [ ]:
train_global_imp(15,300,5,32)
    

In [ ]:
# model definition
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(1894, 100)
        self.act1 = nn.Sigmoid()
        self.hidden2 = nn.Linear(100, 200)
        self.act2 = nn.Sigmoid()
        self.hidden3 = nn.Linear(200, 50)
        self.output= nn.Linear(50, 1)
        self.act_output = nn.Sigmoid()

 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act2(self.hidden3(x))
        x = self.act_output(self.output(x))
        return x

In [ ]:
def num_model(n):
    model_list=[]
    for i in range(n):
        model_list.append(Model())
    return model_list

In [ ]:
def creat_data(n):
    data= pd.read_csv("mail.csv")
    
    
    

    trainloaders=[]
    testloaders = []


        


    
    
    
    X = data['Message']
    y = data['real']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase=True)
    X_train = feature_extraction.fit_transform(X_train)

    X_test = feature_extraction.transform(X_test)

    X_train=X_train.toarray()
    X_test=X_test.toarray()



    X_train=split_array(X_train, n)
 
    y_train=split_array(y_train, n)


    X_test = torch.Tensor(X_test)
    y_test= torch.Tensor(y_test.values)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    X_test=X_test.to(device)
    y_test=y_test.to(device)
    X_test_hold=X_test
    y_test_hold=y_test
    testloaders=[]
    testloaders.append((X_test,y_test))
    for i in range(n):
        X_train[i] = torch.Tensor(X_train[i])
        y_train[i] = torch.Tensor(y_train[i].values)
        X_train[i] = torch.tensor(X_train[i], dtype=torch.float32)
        y_train[i] = torch.tensor(y_train[i], dtype=torch.float32).reshape(-1, 1)

        X_train[i]=X_train[i].to(device)
        y_train[i]=y_train[i].to(device)



        
        hold1=[]
        hold2=[]
        hold1.append((X_train[i],y_train[i]))
        trainloaders.append(hold1)
    
    
    
    combined_testloader = [item for sublist in testloaders for item in sublist]

    return trainloaders,combined_testloader,X_test_hold,y_test_hold

In [ ]:
def train_global(n,k,n_epochs,batch_size):

    mod_list=num_model(n)
    trainloaders,testloaders,x_test,y_test=creat_data(n)


    glob_model=Model()
    acc=[]
    epo=[]
    curr=0
    for i in range(k):

        for i in range(n):
            train(mod_list[i],n_epochs,batch_size,trainloaders[i],0.001)

        epo.append(curr)
        curr=curr+1
        print(curr)

        res_acc=[]  
        res_loss=[]        
        for i in range(n):
            hold=test_whole_data(mod_list[i],x_test,y_test)
            print(hold)
            res_acc.append(hold[1])
            res_loss.append(hold[0])


        avg_value=sum(res_acc)/n
        avg_loss=sum(res_loss)/n
        print(f'avrage value {avg_value}')

        acc.append([avg_value,avg_loss])   

        parameter_lists = [model.parameters() for model in mod_list]

        avg_params = [torch.mean(torch.stack(params), dim=0) for params in zip(*parameter_lists)]

        for glob_param, param in zip(glob_model.parameters(), avg_params):
            glob_param.data.copy_(param)

        mod_list=[]

        for i in range(n):
            mod_list.append(glob_model)







        



    plt.plot(epo,acc,label=('Average Accuracy',"Average Loss"))

    
    plt.ylabel('Accuracy')
    plt.xlabel('Iterations')
    plt.title("Convergence of Federated Learning with 15 Models (5 Epochs, Batch Size 32)")
    plt.legend()
    plt.axes([1, 0.4, 0.2, 0.2])
    plt.boxplot(res_acc)
    plt.xlabel('Accuracy')
    plt.title('Boxplot of Accuracy')
    plt.show()

In [ ]:
train_global(15,100,5,32)